## **IF THIS IS YOUR FIRST TIME STARTING THIS LAB, START AT `part1/part1.ipynb` FIRST**

# EE194 Lab 0: Chisel - Part 0: Chisel Prints, Debugging, Waveforms

## Information/data that is available to you for debugging
* `peek`, `poke` via Chisel (unit) Tests -- We've seen this used throughout this series of exercises to check your designs. 
    * This allows you to drive inputs into the design, explicitly advance clocks, check for output.
    * If you need help with Chisel Tests, look at the "Chisel Test" section in Part 1.
* Print statements
    * Can output helpful information that you aren't explicitly testing for.
    * Can print information during generation (in Scala) or during simulation (in Chisel)
* Waveforms
    * Exported as a VCD file, viewable in [GTKWave](https://gtkwave.sourceforge.net/)

These are just the debugging capability that comes natively with Chisel. Once we move to Chipyard, you will get more debugging capability and debugging will also look a bit different than here. 
* Once we move to Chipyard, you'll get waveform outputs for full RISC-V binaries, compile logs, run logs, full VCS/Verilator capability, support for post synthesis, place & route simulations, etc.

#### Import Chisel dependencies...

In [0]:
interp.configureCompiler(_.settings.processArguments(List("-Wconf:cat=deprecation:s"), true))
interp.load.module(os.Path(s"${System.getProperty("user.dir")}/resource/chisel_deps.sc"))

In [ ]:
import chisel3._
import chisel3.util._
import chiseltest._
import chiseltest.RawTester.test

## Printing

### In Scala (during generation)
* Use `println`
* Harness Scala's string interpolation to print with variables mixed in with text!

In [ ]:
val myVal = 4
println(s"this is $myVal and it is ${myVal.getClass}")

### In Chisel (during simulation, printed once per cycle)
* Use `printf`

2 Ways:
1. C-Style:
```Scala
printf("myVal: %d", myVal)
```
2. Interpolation Style:
```Scala
printf(p"myVal: $myVal")
```

In [ ]:
class MyCounter(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            printf("incrementing from %d\n", count)
//             printf(p"incrementing from $count\n")
            count := count + 1.U
        } .otherwise {
            count := 0.U
            printf("wrapping to        0\n")
        }
    }
    io.out := count
}
// printVerilog(new MyCounter(15))

In [ ]:
test(new MyCounter(3)) { c =>
    c.io.en.poke(1.B)
    c.clock.step(5)
}

### Waveforms
* In theory you shouldn't need this level of debugging in these exercises... but it is here if you need it.
* This will dump a `.vcd` file into `./test_run_dir/chisel_test_<some number>` on each run of the test. Find the latest `chisel_test_<some numer>` folder. That will contain your latest VCD file
* You can then open this `.vcd` file using GTKWave (You want to download the `.vcd` file on to your local computer, install GTKwave locally on your computer and open it that way).

In [ ]:
class MyCounterWaveform(maxVal: Int) extends Module {
    val io = IO(new Bundle {
        val en  = Input(Bool())
        val out = Output(UInt())
    })
    val count = RegInit(0.U(log2Ceil(maxVal+1).W))
    when (io.en) {
        when (count < maxVal.U) {
            printf("incrementing from %d\n", count)
//             printf(p"incrementing from $count\n")
            count := count + 1.U
        } .otherwise {
            count := 0.U
            printf("wrapping to        0\n")
        }
    }
    io.out := count
}
// printVerilog(new MyCounter(15))

In [ ]:
test(new MyCounterWaveform(3), Seq(WriteVcdAnnotation)) { c =>
    c.io.en.poke(1.B)
    c.clock.step(5)
}